# Okavango inundation mapping and precipitation tellaconnection analysis
The following code uses the DWSE watermasking method and Landsat remote sensing data to map inundation extent within the Okavango Delta for the selected months within the study period x-x using Google Earth Engine's Python API. The code also reads in raster data of precipitation from the CHIRPS dataset to calculate monthly precipitation data for the study period in the upper watershed, as well as within a 100 km buffer of the study area.

Author: James (Huck) Rees;
        PhD Student, UCSB Geography
        
Date: February 29, 2024

# Reset code if needed

In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


# Import packages

In [1]:
import geemap
import ee
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
import statistics as stats
import scipy.stats as stats
import json
import folium
from ipyleaflet import Map, DrawControl
from shapely.geometry import shape
from io import StringIO
from shapely.geometry import shape
import os
import rasterio
from rasterio.mask import mask
from shapely.geometry import box
from rasterio.plot import show
from shapely.geometry import mapping
import requests
import os
import gzip
import shutil
from datetime import datetime
import fiona
import re
import matplotlib.dates as mdates
from itertools import cycle
import csv
ee.Initialize()

# Import study regions

## 1. Import study regions

Pathnames to locally stored shapefiles

In [2]:
# Shapefile path
delta_bnd_path = r"C:\Users\huckr\Desktop\UCSB\Okavango\Inundation_Mapping\GIS_Inputs\Okavango_StudAr_OSM_v3.shp"
delta_buff_path = r"C:\Users\huckr\Desktop\UCSB\Okavango\Inundation_Mapping\GIS_Inputs\DeltaBuffer\Delta_100km_Buffer.shp"
US_watershed_path = r"C:\Users\huckr\Desktop\UCSB\Okavango\Inundation_Mapping\GIS_Inputs\Upstream_Watersheds\US_Watersheds_WGS84.shp"

Geemap feature collection of delta boundary

In [3]:
# Load the study area using geopandas
StudArGDF = gpd.read_file(delta_bnd_path)

# Convert geodataframe to geojson then to FeatureCollection
StudAr = geemap.geopandas_to_ee(StudArGDF)

Code for extracting upstream watershed boundaries (shouldn't need to run)

In [19]:
selected_ids = [1041479540, 1041477980]
watershed_fc = ee.FeatureCollection("WWF/HydroATLAS/v1/Basins/level04") \
    .filter(ee.Filter.inList('HYBAS_ID', selected_ids))

def ee_featurecollection_to_geopandas(fc):
    """Converts a ee.FeatureCollection to a GeoPandas GeoDataFrame."""
    # Extract features from the FeatureCollection
    features = fc.getInfo()['features']
    # Convert features to GeoJSON
    geojsons = [feature['geometry'] for feature in features]
    # Convert GeoJSONs to Shapely geometries and create a GeoDataFrame
    geometries = [shape(geojson) for geojson in geojsons]
    gdf = gpd.GeoDataFrame(geometry=geometries)
    gdf['MAIN_BAS'] = [feature['properties']['MAIN_BAS'] for feature in features]
    return gdf

# Convert directly to GeoPandas GeoDataFrame and dissolve by 'MAIN_BAS'
watersheds = ee_featurecollection_to_geopandas(watershed_fc).dissolve(by='MAIN_BAS')

## 2. Source code for DWSE watermasks

In [32]:
def Mndwi(image):
    return(image.normalizedDifference(['Green', 'Swir1']).rename('mndwi'))

def Mbsrv(image):
    return(image.select(['Green']).add(image.select(['Red'])).rename('mbsrv'))

def Mbsrn(image):
    return(image.select(['Nir']).add(image.select(['Swir1'])).rename('mbsrn'))

def Ndvi(image):
    return(image.normalizedDifference(['Nir', 'Red']).rename('ndvi'))

def Awesh(image):
    return(image.expression('Blue + 2.5 * Green + (-1.5) * mbsrn + (-0.25) * Swir2', {
    'Blue': image.select(['Blue']),
    'Green': image.select(['Green']),
    'mbsrn': Mbsrn(image).select(['mbsrn']),
    'Swir2': image.select(['Swir2'])}))

def Dswe(i):
    mndwi = Mndwi(i)
    mbsrv = Mbsrv(i)
    mbsrn = Mbsrn(i)
    awesh = Awesh(i)
    swir1 = i.select(['Swir1'])
    nir = i.select(['Nir'])
    ndvi = Ndvi(i)
    blue = i.select(['Blue'])
    swir2 = i.select(['Swir2'])

    t1 = mndwi.gt(0.124)
    t2 = mbsrv.gt(mbsrn)
    t3 = awesh.gt(0)
    t4 = (mndwi.gt(-0.44)
    .And(swir1.lt(0.09))
    .And(nir.lt(0.15))
    .And(ndvi.lt(0.7)))
    t5 = (mndwi.gt(-0.5)
    .And(blue.lt(0.1))
    .And(swir1.lt(0.3))
    .And(swir2.lt(0.1))
    .And(nir.lt(0.25)))

    t = t1.add(t2.multiply(10)).add(t3.multiply(100)).add(t4.multiply(1000)).add(t5.multiply(10000))

    noWater = (t.eq(0)
    .Or(t.eq(1))
    .Or(t.eq(10))
    .Or(t.eq(100))
    .Or(t.eq(1000)))
    hWater = (t.eq(1111)
    .Or(t.eq(10111))
    .Or(t.eq(11011))
    .Or(t.eq(11101))
    .Or(t.eq(11110))
    .Or(t.eq(11111)))
    mWater = (t.eq(111)
    .Or(t.eq(1011))
    .Or(t.eq(1101))
    .Or(t.eq(1110))
    .Or(t.eq(10011))
    .Or(t.eq(10101))
    .Or(t.eq(10110))
    .Or(t.eq(11001))
    .Or(t.eq(11010))
    .Or(t.eq(11100)))
    pWetland = t.eq(11000)
    lWater = (t.eq(11)
    .Or(t.eq(101))
    .Or(t.eq(110))
    .Or(t.eq(1001))
    .Or(t.eq(1010))
    .Or(t.eq(1100))
    .Or(t.eq(10000))
    .Or(t.eq(10001))
    .Or(t.eq(10010))
    .Or(t.eq(10100)))

    iDswe = (noWater.multiply(0)
    .add(hWater.multiply(1))
    .add(mWater.multiply(2))
    .add(pWetland.multiply(3))
    .add(lWater.multiply(4)))

    return(iDswe.rename(['dswe']))

def ClassifyWaterJones2019(img):
    dswe = Dswe(img)
    waterMask = dswe.eq(1).Or(dswe.eq(2)).Or(dswe.eq(3)).Or(dswe.eq(4)).rename(['waterMask'])
    return(waterMask)

## 3. Create Landsat Image Collection for Study Period

In [33]:
def maskL8sr(image):
    """
    Masks out clouds within the images
    """
    # Bits 3 and 5 are cloud shadow and cloud
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    # Get pixel QA band
    qa = image.select('BQA')
    # Both flags should be zero, indicating clear conditions
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(
        qa.bitwiseAnd(cloudsBitMask).eq(0)
    )
    return image.updateMask(mask)

def getLandsatCollection():
    """
    merge landsat 5, 7, 8 collection 1
    tier 1 SR imageCollections and standardize band names
    """
    # standardize band names
    bn8 = ['B1', 'B2', 'B3', 'B4', 'B6', 'pixel_qa', 'B5', 'B7']
    bn7 = ['B1', 'B1', 'B2', 'B3', 'B5', 'pixel_qa', 'B4', 'B7']
    bn5 = ['B1', 'B1', 'B2', 'B3', 'B5', 'pixel_qa', 'B4', 'B7']
    bns = ['uBlue', 'Blue', 'Green', 'Red', 'Swir1', 'BQA', 'Nir', 'Swir2']

    # create a merged collection from landsat 5, 7, and 8
    ls5 = ee.ImageCollection("LANDSAT/LT05/C01/T1_SR").select(bn5, bns)

    ls7 = (ee.ImageCollection("LANDSAT/LE07/C01/T1_SR")
           .filterDate('1999-04-15', '2003-05-30')
           .select(bn7, bns))

    ls8 = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR").select(bn8, bns)

    merged = ls5.merge(ls7).merge(ls8)

    return(merged)

## 4) Develop water masks and calculate area inundated

In [34]:
def get_maxinundation_mask(start, end, polygon):

    images = ee.List([])
    
    masks = (getLandsatCollection()
        .map(maskL8sr)
        .filterDate(start, end)
        .filterBounds(polygon)
        .map(ClassifyWaterJones2019))

    return masks.max().clip(polygon)

def get_annual_max_inundation_images(start_year, end_year, polygon):
    """
    Create a dictionary of annual maximum inundation images.

    Parameters:
    - start_year: int, the starting year of the period.
    - end_year: int, the ending year of the period.
    - polygon: ee.Geometry.Polygon, the study area.

    Returns:
    A dictionary with years as keys and ee.Image objects representing
    maximum inundation for each year as values.
    """
    annual_images = {}
    for year in range(start_year, end_year + 1):
        start_date = f'{year}-07-01'
        end_date = f'{year}-11-30'
        max_inundation_img = get_maxinundation_mask(start_date, end_date, polygon)
        annual_images[year] = max_inundation_img
    
    return annual_images

def calculate_total_area(start_year, end_year, polygon):
    """
    Calculate the total area (in square kilometers) for each year's image in the provided dictionary, 
    excluding specified years.

    Parameters:
    - start_year: int, the start year for the analysis.
    - end_year: int, the end year for the analysis.
    - polygon: ee.Geometry, the geometry over which to perform the area calculation.

    Returns:
    A list of tuples, where each tuple contains (year, area in square kilometers), excluding specified years.
    """
    # Assuming get_annual_max_inundation_images is defined elsewhere and returns a dictionary
    image_dict = get_annual_max_inundation_images(start_year, end_year, polygon)
    total_areas = []

    # Years to be excluded from the analysis
    excluded_years = set(range(1985, 1989)) | {1993, 2003} | set(range(2009, 2013)) | {2022, 2023}

    for year, image in image_dict.items():
        # Convert year key to integer if it's a string, adjust based on your dictionary keys
        year_int = int(year)

        # Skip years that are in the excluded list
        if year_int in excluded_years:
            continue

        # Use reduceRegion to calculate the sum of pixels within the study area for the current image
        pixel_count = image.reduceRegion(
            reducer=ee.Reducer.sum(),
            geometry=polygon,
            scale=30,  # Adjust this scale if necessary
            maxPixels=1e9,
            bestEffort=True
        ).get('waterMask').getInfo()  # Assuming 'waterMask' is the band with relevant data

        # Convert pixel count to area in square kilometers
        area_km2 = pixel_count * 0.0008474425  # Adjust the multiplication factor if necessary
        
        # Append the year and calculated area to the list
        total_areas.append((year_int, area_km2))
    
    return total_areas

## 5) Create tuples of year and maximum inundation extent 
Very time consuming, do not run unless you have at least 3 minutes per year to run

In [35]:
def write_areas_to_csv(area_tuples, output_path):
    """
    Writes the list of tuples containing year and area information to a CSV file.

    Parameters:
    - area_tuples: List of tuples, where each tuple contains (year, area in square kilometers).
    - output_path: String, the path where the CSV file will be saved.
    """
    # Open the file at the specified output path in write mode
    with open(output_path, 'w', newline='') as csvfile:
        # Create a CSV writer object
        writer = csv.writer(csvfile)
        
        # Write the header row
        writer.writerow(['Year', 'Area (sq km)'])
        
        # Write all tuples from the area_tuples list to the CSV file
        for area_tuple in area_tuples:
            writer.writerow(area_tuple)

In [36]:
MaxInund_DSWE = calculate_total_area(2013, 2016, StudAr)
write_areas_to_csv(MaxInund_DSWE, r"C:\Users\huckr\Desktop\UCSB\Okavango\Inundation_Mapping\Outputs\Test_Landsat_C02_DSWE_Calcs.csv")

# CHIRPS Data Processing and Preliminary Analysis

## 1. Scrape monthly CHIRPS data from online repository
Be cautious with this function. It will take a while to run 

In [10]:
def download_and_extract_chirps(start_year, end_year, target_directory):
    base_url = "https://data.chc.ucsb.edu/products/CHIRPS-2.0/africa_dekad/tifs/"
    
    if not os.path.exists(target_directory):
        os.makedirs(target_directory)

    for year in range(start_year, end_year + 1):
        for month in range(1, 13):
            for dekad in range(1, 4):
                filename = f"chirps-v2.0.{year}.{month:02d}.{dekad}.tif.gz"
                url = f"{base_url}/{filename}"
                
                # Download the file
                print(f"Downloading {url}")
                response = requests.get(url, stream=True)
                if response.status_code == 200:
                    gz_path = os.path.join(target_directory, filename)
                    with open(gz_path, 'wb') as f:
                        f.write(response.content)
                    
                    # Extract the .tif file
                    tif_path = gz_path[:-3]  # Remove the '.gz' extension
                    with gzip.open(gz_path, 'rb') as f_in:
                        with open(tif_path, 'wb') as f_out:
                            shutil.copyfileobj(f_in, f_out)
                    
                    # Optionally, remove the .gz file after extraction
                    os.remove(gz_path)
                else:
                    print(f"Failed to download {url}")

download_and_extract_chirps(1981, 2023, r'C:\Users\huckr\Desktop\UCSB\Okavango\Precip\Dekad')

## 2. Clip rasters to study area boundaries and get precipitation at each dekad time step

In [14]:
def rasterio_clip_and_calculate_mean(shapefile_path, raster_path, output_raster_path):
    with fiona.open(shapefile_path, "r") as shapefile:
        shapes = [feature["geometry"] for feature in shapefile]
    
    with rasterio.open(raster_path) as src:
        out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True, nodata=0)
        out_meta = src.meta
    
    masked_array = np.ma.masked_equal(out_image, 0)
    mean_value = masked_array.mean()
    
    if np.ma.is_masked(mean_value):
        mean_value = np.nan  # Handle completely masked arrays
    
    out_meta.update({"driver": "GTiff", "height": out_image.shape[1], "width": out_image.shape[2], "transform": out_transform})
    with rasterio.open(output_raster_path, "w", **out_meta) as dest:
        dest.write(out_image)
    
    return mean_value

def process_folder(input_folder, shapefile_path, output_folder):
    raster_files = [f for f in os.listdir(input_folder) if f.endswith('.tif')]
    results = []  # Use a list to collect dictionaries

    for raster_file in raster_files:
        raster_path = os.path.join(input_folder, raster_file)
        output_raster_path = os.path.join(output_folder, f"Clipped_{raster_file}")
        
        match = re.search(r'(\d{4})\.(\d{2})\.(\d)\.tif', raster_file)
        if match:
            year, month, dekad = match.groups()
            # Approximate the day based on dekad
            day_map = {'1': '05', '2': '15', '3': '25'}
            approx_day = day_map[dekad]
            date = f"{year}-{month}-{approx_day}"
            
            mean_value = rasterio_clip_and_calculate_mean(shapefile_path, raster_path, output_raster_path)
            
            # Add the constructed date and mean value to the results list
            results.append({"Date": pd.to_datetime(date, format="%Y-%m-%d"), "Year": year, "Month": month, "Dekad": dekad, "Mean pixel value": mean_value})
        else:
            print(f"Filename does not match expected format and will be skipped: {raster_file}")

    # Convert the list of dictionaries to a DataFrame and sort by Date
    df = pd.DataFrame(results).sort_values(by="Date")
    return df

input_folder = r"C:\Users\huckr\Desktop\UCSB\Okavango\Precip\Dekad"
shapefile_path = r"C:\Users\huckr\Desktop\UCSB\Okavango\Inundation_Mapping\GIS_Inputs\Delta_WGS84.shp"
output_folder = r'C:\Users\huckr\Desktop\UCSB\Okavango\Precip\Dekad_Clipped_Delta'
os.makedirs(output_folder, exist_ok=True)

df_results = process_folder(input_folder, shapefile_path, output_folder)
df_results.to_csv(os.path.join(output_folder, "raster_means.csv"), index=False)

Creates dataframe of cumulative precipitation by water year from dataframe of dekad precipitation

In [16]:
def add_cumulative_rainfall(df):
    # Convert 'Date' to datetime if it's not already
    df['Date'] = pd.to_datetime(df['Date'])
    
    # Define a function to determine the water year
    def get_water_year(date):
        if date.month >= 7:
            return date.year + 1  # Water year starts in July
        else:
            return date.year
    
    # Apply the function to create a new 'Water Year' column
    df['Water Year'] = df['Date'].apply(get_water_year)
    
    # Sort the DataFrame by Date to ensure correct cumulative sum calculation
    df = df.sort_values(by='Date')
    
    # Group by 'Water Year' and calculate the cumulative sum of precipitation within each group
    df['Cumulative Rainfall'] = df.groupby('Water Year')['Mean pixel value'].cumsum()
    
    return df

         Date  Year Month Dekad  Mean pixel value  Water Year  Water_Year  \
0  2020-01-05  2020    01     1         71.304936        2020        2020   
1  2020-01-15  2020    01     2         48.632738        2020        2020   
2  2020-01-25  2020    01     3         39.971281        2020        2020   
3  2020-02-05  2020    02     1         51.491661        2020        2020   
4  2020-02-15  2020    02     2         54.472201        2020        2020   
..        ...   ...   ...   ...               ...         ...         ...   
67 2021-11-15  2021    11     2         27.167672        2022        2022   
68 2021-11-25  2021    11     3         23.002383        2022        2022   
69 2021-12-05  2021    12     1         36.905181        2022        2022   
70 2021-12-15  2021    12     2         49.241706        2022        2022   
71 2021-12-25  2021    12     3         22.355078        2022        2022   

    Cumulative Rainfall  
0             71.304936  
1            119.937674

## 3. Calculate total precipitation for the water year

In [11]:
def summarize_precipitation_by_wateryear(results_df):
    # Ensure 'Date' column is of datetime type
    results_df['Date'] = pd.to_datetime(results_df['Date'])

    # Generate 'Water Year' column based on the date
    results_df['Water Year'] = results_df['Date'].apply(lambda x: f"{x.year-1}_{x.year}" if x.month < 7 else f"{x.year}_{x.year+1}")

    # Group by 'Water Year' and sum the 'Mean pixel value' for each group
    summary_df = results_df.groupby('Water Year')['Mean pixel value'].sum().reset_index()

    # Rename the columns for clarity
    summary_df.columns = ['Water Year', 'Total Precipitation (mm)']

    return summary_df

# Example usage:
# Assuming results_df is your original DataFrame with 'Date' and 'Mean pixel value' columns
df_results = df_results
summary_df = summarize_precipitation_by_wateryear(df_results)
print(summary_df)

  Water Year  Total Precipitation (mm)
0  2019_2020                520.658747
1  2020_2021                644.674967
2  2021_2022                216.835164
